# Tutorial 01b: Azure AI Foundry Agent - Enterprise-Ready Travel Assistant

## 🎯 Learning Objectives
By the end of this notebook, you will:
- Understand what Azure AI Foundry (formerly Azure AI Studio) is
- Learn the differences between OpenAI and Azure AI Foundry agents
- Create agents using Azure AI Foundry with enterprise features
- Work with Azure authentication and credentials
- Understand when to use Azure AI Foundry vs. direct OpenAI

## 📚 Key Concepts

### What is Azure AI Foundry?
Azure AI Foundry is Microsoft's enterprise AI platform that provides:
- **Managed AI Models** - Host and scale OpenAI and other models
- **Enterprise Security** - VNET, RBAC, managed identities
- **Built-in Tools** - Code interpreter, file search, Bing grounding
- **Content Safety** - Automatic content filtering and safety checks
- **Monitoring** - Built-in logging, metrics, and tracing
- **Compliance** - Meet enterprise compliance requirements

### OpenAI vs Azure AI Foundry

| Feature | OpenAI Direct | Azure AI Foundry |
|---------|--------------|------------------|
| **Setup** | Simple API key | Azure subscription + project |
| **Security** | API key only | Enterprise security (RBAC, VNET) |
| **Tools** | Function calling | Functions + Code Interpreter + File Search + Bing |
| **Monitoring** | External | Built-in Azure Monitor |
| **Cost** | Pay-per-token | Pay-per-token + Azure overhead |
| **Compliance** | Limited | Full Azure compliance |
| **Best For** | Prototypes, startups | Enterprise, production |

---

## Step 1: Prerequisites

Before starting, you need:

1. **Azure Subscription** - [Get a free account](https://azure.microsoft.com/free/)
2. **Azure AI Foundry Project** - [Create one here](https://ai.azure.com/)
3. **Azure CLI** - For authentication
4. **Model Deployment** - Deploy a model in your project

### Quick Setup Guide

```bash
# 1. Install Azure CLI (if not already installed)
# macOS: brew install azure-cli
# Windows: Download from https://aka.ms/installazurecliwindows

# 2. Login to Azure
az login

# 3. Install the Azure AI package
pip install agent-framework-azure-ai
```

## Step 2: Setup and Imports

In [ ]:
# Install required packages (uncomment if needed)
# !pip install agent-framework-azure-ai azure-identity python-dotenv

In [6]:
import asyncio
import os
from random import randint, choice
from typing import Annotated

from agent_framework import ChatAgent
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential, DefaultAzureCredential
from pydantic import Field
from dotenv import load_dotenv

load_dotenv()
print("✅ Imports successful!")

✅ Imports successful!


## Step 3: Configure Azure AI Foundry Connection

You need two pieces of information from your Azure AI Foundry project:

1. **Project Endpoint** - Found in your project overview
2. **Model Deployment Name** - The name of your deployed model

### How to Get These Values:

1. Go to [Azure AI Foundry](https://ai.azure.com/)
2. Open your project
3. **Project Endpoint**: Go to "Project Settings" → Copy the endpoint URL
4. **Model Deployment**: Go to "Deployments" → Find your deployed model name

### Configuration Options:

In [ ]:

project_endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
model_deployment = os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME")

if project_endpoint and model_deployment:
    print("✅ Configuration loaded from environment variables")
    print(f"   Endpoint: {project_endpoint}")
    print(f"   Model: {model_deployment}")
else:
    print("⚠️  Please set AZURE_AI_PROJECT_ENDPOINT and AZURE_AI_MODEL_DEPLOYMENT_NAME")
    print("   You can set them in your .env file or directly in this notebook")
    

✅ Configuration loaded from environment variables
   Endpoint: https://a35tm-aiagents-oai.services.ai.azure.com/api/projects/a35tm-aiagents-ai-project
   Model: gpt-4.1


## Step 4: Understanding Azure Authentication

Azure AI Foundry uses **Azure Active Directory** for authentication (not API keys!).

### Authentication Options:

1. **AzureCliCredential** (easiest for development)
   - Uses your `az login` session
   - Great for local development

2. **DefaultAzureCredential** (recommended for production)
   - Tries multiple authentication methods
   - Works in Azure, local development, and CI/CD

3. **ManagedIdentityCredential** (for Azure-hosted apps)
   - Uses Azure Managed Identity
   - No secrets to manage!

In [8]:
# Check if authenticated
async def check_auth():
    try:
        async with AzureCliCredential() as credential:
            token = await credential.get_token("https://management.azure.com/.default")
            if token:
                print("✅ Successfully authenticated with Azure")
                return True
    except Exception as e:
        print(f"❌ Authentication failed: {e}")
        print("   Run 'az login' in your terminal first")
        return False

await check_auth()

✅ Successfully authenticated with Azure


True

## Step 5: Create Your First Azure AI Agent

Let's create a Travel Assistant using Azure AI Foundry.

Notice the key differences from OpenAI:
- Use `AzureAIAgentClient` instead of `OpenAIChatClient`
- Pass `async_credential` for authentication
- Agent is automatically created in Azure AI Foundry
- Use `async with` for proper cleanup

In [15]:
# Create a simple agent
async def create_basic_agent():
    # Use async with for automatic cleanup
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(
            async_credential=credential,
            # project_endpoint and model_deployment_name can be omitted if set in env vars
        ).create_agent(
            name="AzureTravelAssistant",
            instructions="""
            You are a friendly and knowledgeable travel assistant powered by Azure AI.
            Help users plan trips, answer questions about destinations, and provide travel advice.
            
            Be enthusiastic, specific, and consider factors like budget and time of year.
            """
        ) as agent,
    ):
        # Ask a question
        query = "What are the best destinations for a winter vacation in Europe?"
        print(f"User: {query}\n")
        
        response = await agent.run(query)
        print(f"Agent: {response}")
        
        return response

# Run it
if project_endpoint and model_deployment:
    await create_basic_agent()
else:
    print("⚠️  Please configure your Azure AI project first")

User: What are the best destinations for a winter vacation in Europe?

Agent: Europe is magical in winter, whether you crave snow-capped mountains, festive city lights, or mild Mediterranean escapes! Here are some top destinations, tailored to different travel styles and budgets:

**1. For Winter Wonderland & Skiing Lovers:**
- **The Swiss Alps (Zermatt, St. Moritz, Verbier):** Iconic snowy peaks, world-class skiing, cozy chalets. Zermatt offers stunning views of the Matterhorn!
- **Austrian Alps (Innsbruck, Salzburg region):** Charming villages, Christmas markets, and diverse ski resorts for all levels.
- **French Alps (Chamonix, Annecy):** Excellent slopes and picturesque lakeside towns.

**2. For Festive City Escapes:**
- **Vienna, Austria:** Famous for enchanting Christmas markets, imperial palaces, and traditional cafés.
- **Prague, Czech Republic:** Fairytale Old Town, sparkling holiday lights, and hearty winter foods.
- **Strasbourg & Colmar, France:** Renowned as the “Capital o

## Step 6: Add Tools to Azure AI Agent

Azure AI agents support the same function tools as OpenAI agents.

In [10]:
# Define tools
def get_weather(
    location: Annotated[str, Field(description="City or location name")],
) -> str:
    """Get current weather for a location."""
    conditions = ["sunny", "partly cloudy", "cloudy", "rainy", "stormy"]
    temp = randint(10, 35)
    condition = choice(conditions)
    return f"The weather in {location} is {condition} with {temp}°C."

def convert_currency(
    amount: Annotated[float, Field(description="Amount to convert")],
    from_currency: Annotated[str, Field(description="Source currency (USD, EUR, GBP, etc.)")],
    to_currency: Annotated[str, Field(description="Target currency")],
) -> str:
    """Convert currency amounts."""
    rates = {"USD": 1.0, "EUR": 0.92, "GBP": 0.79, "JPY": 149.50}
    from_currency, to_currency = from_currency.upper(), to_currency.upper()
    
    if from_currency not in rates or to_currency not in rates:
        return f"Sorry, unsupported currency: {from_currency} or {to_currency}"
    
    converted = (amount / rates[from_currency]) * rates[to_currency]
    return f"{amount} {from_currency} = {converted:.2f} {to_currency}"

print("✅ Tools defined")

✅ Tools defined


In [11]:
# Create agent with tools
async def create_agent_with_tools():
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(
            async_credential=credential,
        ).create_agent(
            name="AzureTravelAssistantWithTools",
            instructions="""
            You are a travel assistant with access to real-time tools.
            Use the weather tool to check conditions and currency converter for budgeting.
            """,
            tools=[get_weather, convert_currency]  # Add tools here
        ) as agent,
    ):
        query = "What's the weather in Barcelona? Also, how much is 500 USD in Euros?"
        print(f"User: {query}\n")
        
        response = await agent.run(query)
        print(f"Agent: {response}")

if project_endpoint and model_deployment:
    await create_agent_with_tools()
else:
    print("⚠️  Please configure your Azure AI project first")

User: What's the weather in Barcelona? Also, how much is 500 USD in Euros?

Agent: The current weather in Barcelona is cloudy with a temperature of 11°C.

As for currency exchange, 500 USD is approximately 460.00 Euros. If you need more weather details or currency tips, let me know!
Agent: The current weather in Barcelona is cloudy with a temperature of 11°C.

As for currency exchange, 500 USD is approximately 460.00 Euros. If you need more weather details or currency tips, let me know!


## Step 7: Streaming Responses

Azure AI agents support streaming for better user experience.

In [12]:
async def streaming_example():
    print("=== Streaming Response ===\n")
    
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(
            async_credential=credential,
        ).create_agent(
            name="StreamingTravelAgent",
            instructions="You are a helpful travel assistant.",
            tools=[get_weather]
        ) as agent,
    ):
        query = "Plan a 3-day itinerary for Paris, including weather info."
        print(f"User: {query}\n")
        print("Agent: ", end="", flush=True)
        
        # Stream the response
        async for chunk in agent.run_stream(query):
            if chunk.text:
                print(chunk.text, end="", flush=True)
        
        print("\n")  # New line at the end

if project_endpoint and model_deployment:
    await streaming_example()
else:
    print("⚠️  Please configure your Azure AI project first")

=== Streaming Response ===

User: Plan a 3-day itinerary for Paris, including weather info.

Agent: Here’s aHere’s a 3-day Paris 3-day Paris itinerary, itinerary, with today’s weather with today’s weather info:

 info:

Current Weather:Current Weather: Sunny,  Sunny, 1515°C (great°C (great for exploring outdoors for exploring outdoors!)

Day!)

Day 1: Classic Paris S 1: Classic Paris Sights
ights
- Morning:- Morning: Visit the Visit the Eiffel Tower ( Eiffel Tower (go early togo early to avoid crowds avoid crowds), stroll Champ), stroll Champ de de Mars.
 Mars.
-- Afternoon: Walk along River Seine to the Musée d'Orsay. Enjoy impressionist art.
 Afternoon: Walk along River Seine to the Musée d'Orsay. Enjoy impressionist art.
- Evening: Dinner in Saint-Germain-des-Prés, then- Evening: Dinner in Saint-Germain-des-Prés, then admire admire the city the city views from views from Pont Alexandre Pont Alexandre III.

 III.

Day Day 2: Culture2: Culture & Charming & Charming Neighborhoods
 Neig

## Step 8: Explicit Configuration

You can also configure everything explicitly without environment variables.

In [13]:
async def explicit_config_example():
    # Explicit configuration (useful for multiple projects)
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(
            async_credential=credential,
            project_endpoint="https://a35tm-aiagents-oai.services.ai.azure.com/api/projects/a35tm-aiagents-ai-project",  # Your endpoint
            model_deployment_name="gpt-4.1",  # Your deployment
        ).create_agent(
            name="ExplicitConfigAgent",
            instructions="You are a helpful assistant.",
        ) as agent,
    ):
        response = await agent.run("Hello!")
        print(response)

# Uncomment to run with your actual values
# await explicit_config_example()

## 🔍 Understanding Azure AI Agent Features

### Automatic Lifecycle Management

When you use `async with`, the agent is:
1. **Created** automatically in Azure AI Foundry
2. **Used** for your queries
3. **Deleted** automatically when done

### Reusing Agents

For production, you might want to reuse agents:

```python
# Create once
agent_client = AzureAIAgentClient(credential=credential)
agent_id = await agent_client.create_agent_async(name="MyAgent", ...)

# Reuse many times
agent = ChatAgent(chat_client=agent_client, agent_id=agent_id)
```

See the samples folder for `azure_ai_with_existing_agent.py` example.

## Step 9: Side-by-Side Comparison

Let's compare OpenAI vs Azure AI Foundry agents side by side.

In [ ]:
# First, let's define a common query and tool
test_query = "What are the top 3 destinations for beach lovers?"

# OpenAI Version
async def test_openai():
    from agent_framework.openai import OpenAIChatClient
    from agent_framework.azure import AzureOpenAIChatClient  # Added missing import
    
    print("=== OpenAI Agent ===")
    agent = ChatAgent(
        chat_client=AzureOpenAIChatClient(endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"), api_key=os.getenv("AZURE_OPENAI_API_KEY"),deployment_name="gpt-4.1"),  # Use Azure OpenAI if needed
        name="OpenAITravel",
        instructions="You are a travel assistant."
    )
    response = await agent.run(test_query)
    print(f"Response: {response[:200]}...\n")

# Azure AI Version  
async def test_azure():
    print("=== Azure AI Agent ===")
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(async_credential=credential).create_agent(
            name="AzureTravel",
            instructions="You are a travel assistant."
        ) as agent,
    ):
        response = await agent.run(test_query)
        print(f"Response: {response[:200]}...\n")

# Run both (if configured)
if os.getenv("OPENAI_API_KEY"):
    await test_openai()
else:
    print("OpenAI: ⚠️  OPENAI_API_KEY not set\n")

if project_endpoint and model_deployment:
    await test_azure()
else:
    print("Azure AI: ⚠️  Azure AI not configured\n")

## 💡 Key Takeaways

### Use Azure AI Foundry When You Need:
- ✅ **Enterprise security** - RBAC, VNET, compliance
- ✅ **Built-in monitoring** - Azure Monitor integration
- ✅ **Content safety** - Automatic filtering
- ✅ **Advanced tools** - Code interpreter, file search, Bing grounding
- ✅ **Team collaboration** - Shared projects and resources

### Use Direct OpenAI When You Need:
- ✅ **Quick prototyping** - Faster setup
- ✅ **Simple projects** - No Azure overhead
- ✅ **Cost optimization** - No Azure infrastructure costs
- ✅ **Maximum flexibility** - Direct API access

### Code Differences:

| Aspect | OpenAI | Azure AI Foundry |
|--------|---------|------------------|
| Import | `OpenAIChatClient` | `AzureAIAgentClient` |
| Auth | API key | Azure credential |
| Setup | Instant | Requires Azure project |
| Context Manager | Optional | Recommended (`async with`) |

### Best Practice:
- **Develop** with OpenAI (faster iteration)
- **Deploy** with Azure AI (production-ready)

## 🎯 Practice Exercises

1. **Create an Azure AI agent** with your own instructions
2. **Add custom tools** for your use case
3. **Compare costs** - Check Azure AI pricing vs OpenAI
4. **Explore the portal** - Look at your project in ai.azure.com
5. **Try different auth methods** - Test DefaultAzureCredential

In [ ]:
# Your practice code here!
# Try creating an agent with different tools or instructions

## 🚀 Advanced Azure AI Features (Coming in Later Tutorials)

Azure AI Foundry offers additional capabilities we'll explore later:

- **Code Interpreter** - Execute Python code safely (Tutorial 06)
- **File Search** - Search through uploaded documents (Tutorial 06)
- **Bing Grounding** - Real-time web search (Tutorial 06)
- **Content Safety** - Automatic harmful content filtering (Tutorial 05)
- **Monitoring** - Azure Monitor integration (Tutorial 11)
- **Managed Identity** - Secure authentication without keys (Tutorial 12)

Check out the samples folder for examples:
- `agent-framework/python/samples/getting_started/agents/azure_ai/`

## 📚 What's Next?

Now that you understand both OpenAI and Azure AI Foundry agents:

**Continue to Tutorial 02** → `02_agent_with_tools.ipynb`

Or explore Azure-specific features:
- Check out `/agent-framework/python/samples/getting_started/agents/azure_ai/`
- Try the Bing grounding example
- Experiment with file search

---

### Quick Reference

**Environment Variables for Azure AI:**
```bash
AZURE_AI_PROJECT_ENDPOINT=https://your-project.api.azureml.ms
AZURE_AI_MODEL_DEPLOYMENT_NAME=gpt-4o
```

**Basic Azure AI Agent:**
```python
async with (
    AzureCliCredential() as credential,
    AzureAIAgentClient(async_credential=credential).create_agent(
        name="MyAgent",
        instructions="Your instructions",
        tools=[your_tool]
    ) as agent,
):
    response = await agent.run("Your query")
```